In [1]:
from backtesting import set_bokeh_output
set_bokeh_output(notebook=False)

# !python3 -m pip install bokeh==2.4.3

In [2]:
from backtesting import Backtest, Strategy
from strategies import MultipleBasic, SmaCross, BinaryBasic
from sqlalchemy import create_engine, text
from config import *
import pandas as pd
from datetime import timedelta
from tqdm import tqdm
import sys
sys.path.insert(1, '/home/kenny/algotrading/model_training')
from dataset import Data

In [3]:
url = DATABASE_URL_1HOUR
td_days = 0

In [4]:
engine = create_engine(url=url)

with engine.connect() as connection:
        # query = "SELECT name FROM sqlite_schema WHERE type='table'"
        # symbols = [item[0] for item in connection.execute(text(query)).fetchall()]
        # for symbol in symbols:
        symbol = 'BTCUSDT'
        qry = f"SELECT * FROM '{symbol}' WHERE Open_Time < '{pd.to_datetime('today') - timedelta(days = td_days)}'"
        data = pd.DataFrame(connection.execute(text(qry)))
        data['Open_Time'] = pd.to_datetime(data['Open_Time'])
        data.set_index("Open_Time", inplace=True)
        # data.index = pd.to_datetime(data.index)
        # data.drop(['Close_Time'], axis=1, inplace=True)
connection.close()

In [5]:
# df = Data(data)
# df.create_features()
# df.create_target(task_type='classification')
# df.df

In [6]:
# data.to_csv('example.csv')
# data['Open_Time'] = pd.to_datetime(data['Open_Time'])
# data.set_index("Open_Time", inplace=True)

In [7]:
cash = 10000
commission = .001
bt = Backtest(data, MultipleBasic, cash=cash, commission=commission, margin=.05)
output = bt.run()
# stats = bt.optimize(price_delta=[0.002, 0.004, 0.001],
#                   maximize='Sharpe Ratio')


In [8]:
output

Start                     2023-02-22 07:00:00
End                       2023-02-27 06:00:00
Duration                      4 days 23:00:00
Exposure Time [%]                   98.333333
Equity Final [$]                   2313.62357
Equity Peak [$]                   16451.64187
Return [%]                         -76.863764
Buy & Hold Return [%]               -2.836255
Return (Ann.) [%]                      -100.0
Volatility (Ann.) [%]                     0.0
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -86.229583
Avg. Drawdown [%]                  -33.525025
Max. Drawdown Duration        4 days 05:00:00
Avg. Drawdown Duration        1 days 13:00:00
# Trades                                   88
Win Rate [%]                        44.318182
Best Trade [%]                       0.409721
Worst Trade [%]                     -0.610677
Avg. Trade [%]                    

In [9]:
bt.plot()

Row(id='1240', ...)

In [5]:
import sys
sys.path.insert(1, '/home/kenny/algotrading/model_training')
from dataset import Data

In [6]:
trans = Data(data)
trans.df

,Open,High,Low,Close,Volume,Close_Time
Open_Time,,,,,,
2023-01-10 22:25:00,17438.09,17439.45,17435.26,17436.41,80.09156,2023-01-10 22:25:59.999000
2023-01-10 22:26:00,17436.41,17437.01,17426.76,17428.62,197.24285,2023-01-10 22:26:59.999000
2023-01-10 22:27:00,17428.62,17429.90,17419.92,17425.81,408.72043,2023-01-10 22:27:59.999000
2023-01-10 22:28:00,17425.81,17433.30,17422.76,17432.94,124.77102,2023-01-10 22:28:59.999000
2023-01-10 22:29:00,17432.94,17439.24,17432.47,17438.11,97.83412,2023-01-10 22:29:59.999000
...,...,...,...,...,...,...
2023-01-11 17:54:00,17384.70,17385.32,17381.03,17382.63,92.29850,2023-01-11 17:54:59.999000
2023-01-11 17:55:00,17382.94,17387.48,17382.35,17384.71,95.91164,2023-01-11 17:55:59.999000
2023-01-11 17:56:00,17385.19,17385.45,17378.45,17379.85,97.12869,2023-01-11 17:56:59.999000


In [7]:
trans.create_target(task_type='classification')

In [10]:
set(trans.df['Target'])

{-1.0, 0.0, 1.0}

In [58]:
y = trans.df.Close.pct_change(1).shift(1)

percent = .0005

y[y.between(-percent, percent)] = 0
y[y > 0] = 1
y[y < 0] = -1

In [63]:
trans.df['Target'] = y
trans.df['Target'].fillna(trans.df['Target'].mode(), inplace=True)

In [73]:
trans.df['Target'].fillna(trans.df['Target'].mode().values[0], inplace=True)
trans.df

,Open,High,Low,Close,Volume,Close_Time,Target
Open_Time,,,,,,,
2023-01-10 22:25:00,17438.09,17439.45,17435.26,17436.41,80.09156,2023-01-10 22:25:59.999000,0.0
2023-01-10 22:26:00,17436.41,17437.01,17426.76,17428.62,197.24285,2023-01-10 22:26:59.999000,0.0
2023-01-10 22:27:00,17428.62,17429.90,17419.92,17425.81,408.72043,2023-01-10 22:27:59.999000,0.0
2023-01-10 22:28:00,17425.81,17433.30,17422.76,17432.94,124.77102,2023-01-10 22:28:59.999000,0.0
2023-01-10 22:29:00,17432.94,17439.24,17432.47,17438.11,97.83412,2023-01-10 22:29:59.999000,0.0
...,...,...,...,...,...,...,...
2023-01-11 17:45:00,17386.16,17389.19,17384.96,17385.16,101.58079,2023-01-11 17:45:59.999000,0.0
2023-01-11 17:46:00,17385.16,17386.79,17381.28,17385.94,124.04388,2023-01-11 17:46:59.999000,0.0
2023-01-11 17:47:00,17385.95,17388.35,17383.00,17383.58,68.14626,2023-01-11 17:47:59.999000,0.0


In [42]:
trans.df['Target'] = y

In [48]:
trans.df

,Open,High,Low,Close,Volume,Close_Time,Target
Open_Time,,,,,,,
2023-01-10 22:25:00,17438.09,17439.45,17435.26,17436.41,80.09156,2023-01-10 22:25:59.999000,NaN
2023-01-10 22:26:00,17436.41,17437.01,17426.76,17428.62,197.24285,2023-01-10 22:26:59.999000,NaN
2023-01-10 22:27:00,0.00,0.00,0.00,0.00,0.00000,0,0.0
2023-01-10 22:28:00,0.00,0.00,0.00,0.00,0.00000,0,0.0
2023-01-10 22:29:00,0.00,0.00,0.00,0.00,0.00000,0,0.0
...,...,...,...,...,...,...,...
2023-01-11 17:44:00,0.00,0.00,0.00,0.00,0.00000,0,0.0
2023-01-11 17:45:00,0.00,0.00,0.00,0.00,0.00000,0,0.0
2023-01-11 17:46:00,0.00,0.00,0.00,0.00,0.00000,0,0.0


In [10]:
trans.df

,Open,High,Low,Close,Volume,Close_Time,RSI_14,SMA_3,EMA_3,SMA_5,...,high_3,low_3,close_4,open_4,high_4,low_4,close_5,open_5,high_5,low_5
Open_Time,,,,,,,,,,,,,,,,,,,,,
2023-01-11 00:04:00,17439.89,17440.66,17438.57,17439.49,51.14619,2023-01-11 00:04:59.999000,57.329890,17438.893333,17439.424902,17439.450,...,17442.82,17438.63,17441.05,17440.64,17442.81,17437.55,17440.66,17439.00,17441.58,17437.83
2023-01-11 00:05:00,17439.03,17441.31,17436.71,17440.50,109.13185,2023-01-11 00:05:59.999000,58.943118,17440.046667,17439.962451,17439.340,...,17442.34,17436.00,17439.52,17440.94,17442.82,17438.63,17441.05,17440.64,17442.81,17437.55
2023-01-11 00:06:00,17440.50,17442.39,17439.07,17439.72,109.48042,2023-01-11 00:06:59.999000,57.146244,17439.903333,17439.841225,17439.380,...,17440.25,17436.23,17437.04,17439.77,17442.34,17436.00,17439.52,17440.94,17442.82,17438.63
2023-01-11 00:07:00,17439.72,17440.53,17435.85,17437.72,88.37033,2023-01-11 00:07:59.999000,52.709224,17439.313333,17438.780613,17439.516,...,17440.66,17438.57,17440.15,17437.04,17440.25,17436.23,17437.04,17439.77,17442.34,17436.00
2023-01-11 00:08:00,17437.72,17439.11,17433.86,17436.01,96.66460,2023-01-11 00:08:59.999000,49.192386,17437.816667,17437.395306,17438.688,...,17441.31,17436.71,17439.49,17439.89,17440.66,17438.57,17440.15,17437.04,17440.25,17436.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-02-09 22:20:00,21891.71,21904.59,21850.32,21892.95,258.64371,2023-02-09 22:20:59.999000,53.860101,21900.910000,21900.425972,21916.140,...,21955.00,21914.02,21927.68,21906.87,21950.95,21905.17,21907.91,21890.23,21908.90,21885.93
2023-02-09 22:21:00,21892.94,21893.45,21874.39,21888.69,109.54240,2023-02-09 22:21:59.999000,52.675068,21890.490000,21894.557986,21908.342,...,21955.14,21910.41,21950.29,21927.68,21955.00,21914.02,21927.68,21906.87,21950.95,21905.17
2023-02-09 22:22:00,21888.69,21897.27,21883.98,21890.01,75.06506,2023-02-09 22:22:59.999000,53.019993,21890.550000,21892.283993,21896.286,...,21921.47,21869.00,21919.95,21951.96,21955.14,21910.41,21950.29,21927.68,21955.00,21914.02
